# Playground

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append("./code")

In [13]:
import xml.etree.ElementTree as ET
import pandas as pd
from collections import defaultdict
import os

import logging
logging.basicConfig(format="%(asctime)s - %(levelname)s: %(message)s", level=logging.INFO, datefmt="%I:%M:%S")


from utils import *

In [14]:
REPORT_PATH = "./data/raw/reports"

In [20]:
def load_reports(path):
    reportFeatures = defaultdict(list)

    filenames = get_filenames(path)
    logging.info(f"found {len(filenames)} reports.")

    for filename in filenames:
        tree = ET.parse(os.path.join(REPORT_PATH, filename))
        root = tree.getroot()

        medlineCitationTag = root.find("MedlineCitation")
        assert medlineCitationTag != None, "Failed to find MedlineCitation tag"

        articleTag = medlineCitationTag.find("Article")
        assert articleTag != None, "Failed to find Article tag"

        abstractTag = articleTag.find("Abstract")
        assert abstractTag != None, "Failed to find Abstract tag"

        parentImageTags = root.findall("parentImage")

        for parentImageTag in parentImageTags:
            reportFeatures["image_name"].append(parentImageTag.attrib["id"])

            for reportCategory in abstractTag:
                feature = reportCategory.attrib["Label"].lower()
                reportFeatures[feature].append(reportCategory.text)

    return pd.DataFrame(reportFeatures)

In [21]:
metadata = load_reports(REPORT_PATH)
metadata.describe()

11:38:34 - INFO: found 3955 reports.


,image_name,comparison,indication,findings,impression
count,7470,6313,7311,6473,7418
unique,7470,394,2454,2553,1770
top,CXR82_IM-2350-3001,None.,Chest pain,The heart is normal in size. The mediastinum i...,No acute cardiopulmonary abnormality.
freq,1,1592,253,98,586


In [22]:
metadata.head()

,image_name,comparison,indication,findings,impression
0,CXR82_IM-2350-3001,Chest x-XXXX XXXX,PAIN IN THORACIC SPINE Pain started in leg are...,"No airspace disease, effusion or noncalcified ...",No acute cardiopulmonary abnormality.
1,CXR82_IM-2350-4001,Chest x-XXXX XXXX,PAIN IN THORACIC SPINE Pain started in leg are...,"No airspace disease, effusion or noncalcified ...",No acute cardiopulmonary abnormality.
2,CXR173_IM-0481-1001,None,Chest pain,Low lung volumes. Cardiomediastinal silhouette...,Low lung volumes. No acute cardiopulmonary fin...
3,CXR173_IM-0481-2001,None,Chest pain,Low lung volumes. Cardiomediastinal silhouette...,Low lung volumes. No acute cardiopulmonary fin...
4,CXR3676_IM-1829-0001-0001,Comparison is XXXX to XXXX examination dated X...,XXXX-year-old male with XXXX.,The cardiomediastinal silhouette appears irreg...,1. Redemonstration of diffuse bilateral pulmon...
